In [ ]:
# Ignore warnings for cleaner logs
import warnings
warnings.simplefilter("ignore")

In [ ]:
# Making Necessary Imports
import numpy as np
import pandas as pd
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import librosa
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm

2025-12-16 19:05:36.728284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765911936.891684      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765911936.941773      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
# Defining the model for Speech Extraction
model_id = "openai/whisper-large-v3-turbo"

# Defining the Device and Data Type
device = "cuda"
torch_dtype = torch.float16

# Retrieving the Audio Processor and Model
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True
).to(device)

model.eval();

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
# Reading CSVs
train = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/train.csv')
test = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/test.csv')

In [ ]:
# Adding complete path
train['path'] = '/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train/' + train['filename'] + '.wav'
test['path'] = '/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/' + test['filename'] + '.wav'

In [ ]:
# Defining the batch transcribe function
def batch_transcribe(audio_paths, batch_size=4, beam_size=10):
    results = []
    model.eval()
    for i in tqdm(range(0, len(audio_paths), batch_size), desc='Transcribing: '):
        batch = audio_paths[i:i+batch_size]
        # Loading the audios in a batch
        audio_list = []
        for p in batch:
            audio, _ = librosa.load(p, sr=16000)
            audio_list.append(audio)   # keep as numpy

        # Processing the audio inputs
        inputs = processor(
            audio_list,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True          
        ).to(device)

        # Ensuring the inputs are in FP16
        inputs.input_features = inputs.input_features.half()
        
        # Obtaining transcript
        with torch.no_grad():
            generated_ids = model.generate(
                inputs.input_features,
                num_beams=beam_size,
                language="en",
                task="transcribe"
            )

        # Decoding the tokens to generate text
        texts = processor.batch_decode(
            generated_ids,
            skip_special_tokens=True
        )
        results.extend(texts)

    return results

In [ ]:
# Obtaining the train and test transcript
train['text'] = batch_transcribe(
    list(train['path']),
    batch_size=4,
    beam_size=10
)

test['text'] = batch_transcribe(
    list(test['path']),
    batch_size=4,
    beam_size=10
)

Transcribing:   0%|          | 0/103 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcribing:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Saving the train and test transcripts
train.to_csv('train_updated.csv', index=False)
test.to_csv('test_updated.csv', index=False)